youtube 영상 wav로 다운받기 
<br>
https://4kdownload.to/ko/youtube-wav-downloader

In [1]:
import librosa
import numpy as np

# 오디오 파일 로드
y, sr = librosa.load("beethoven2.mp3")
title = "beethoven"

# 볼륨(Amplitude) 추출
amplitude = np.abs(librosa.stft(y)).mean(axis=1)

# 비트 추출
tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
beat_times = librosa.frames_to_time(beat_frames, sr=sr)

# JSON 데이터 생성 (예: 진동 시간과 강도)
vibration_data = [
    {"time": round(time, 2), "intensity": int(amp / max(amplitude) * 255)} 
    for time, amp in zip(beat_times, amplitude)
]

# JSON 파일로 저장
import json
with open(f"{title}_resultmp3.json", "w") as f:
    json.dump(vibration_data, f)


In [1]:
# 안드용 parallel  data.json 생성 코드

import librosa
import numpy as np
from time import time
import json

t1 = time()
y, sr = librosa.load("beethoven.wav")
title = "beethoven"

# 볼륨(Amplitude) 추출
amplitude = np.abs(librosa.stft(y)).mean(axis=1)

# 비트 추출
tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
beat_times = librosa.frames_to_time(beat_frames, sr=sr)

# JSON 데이터 생성 (예: 진동 시간과 강도)
vibration_data = [
    {"time": [round(time, 2) for time in beat_times], "intensity": [int(amp / max(amplitude) * 255) for amp in amplitude]} 
]

# JSON 파일로 저장

with open(f"{title}_result.json", "w") as f:
    json.dump(vibration_data, f)
    t2 = time()
    print(f'{t2-t1} 초 소요')

3.499215841293335 초 소요


In [4]:
# 코드 최적화 

import librosa
import numpy as np
import json
from time import time

t1 = time()
# 오디오 파일 로드
y, sr = librosa.load("beethoven.wav")
title = "beethoven"


# 볼륨(Amplitude) 추출
amplitude = np.abs(librosa.stft(y)).mean(axis=1)

# 비트 추출
tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
beat_times = np.round(librosa.frames_to_time(beat_frames, sr=sr), 2)  # 반올림을 바로 적용

# 강도 정규화 및 클리핑 (벡터화 연산 사용)
max_amplitude = max(amplitude)
intensities = np.clip((amplitude / max_amplitude * 255), 0, 255).astype(int)

# JSON 데이터 생성
vibration_data = [{
    "time": beat_times.tolist(),
    "intensity": intensities.tolist()
}]

# JSON 파일로 저장
with open(f"{title}_result2.json", "w") as f:
    json.dump(vibration_data, f)
    t2 = time()
    print(f'{t2-t1} 초 소요')


1.2260160446166992 초 소요


In [3]:
# 코드 최적화 최종본

import librosa
import numpy as np
import json
from time import time

t1 = time()
# 오디오 파일 로드
y, sr = librosa.load("beethoven2.mp3")
title = "beethoven"


# 볼륨(Amplitude) 추출
amplitude = np.abs(librosa.stft(y)).mean(axis=1)

# 비트 추출
tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
beat_times = np.round(librosa.frames_to_time(beat_frames, sr=sr), 2)  # 반올림을 바로 적용

# 강도 정규화 및 클리핑 (벡터화 연산 사용)
max_amplitude = max(amplitude)
intensities = np.clip((amplitude / max_amplitude * 255), 0, 255).astype(int)

# JSON 데이터 생성
vibration_data = [{
    "time": beat_times.tolist(),
    "intensity": intensities.tolist()
}]

# JSON 파일로 저장
with open(f"{title}_result2.json", "w") as f:
    json.dump(vibration_data, f)
    t2 = time()
    print(f'{t2-t1} 초 소요')

24.09607172012329 초 소요


In [ ]:
# 유튜브 음원 추출 & 코드 최적화
import pymysql
import librosa
import yt_dlp
import numpy as np
import json
import os
from time import time

t1 = time()

# DB 연결
conn = pymysql.connect(host='127.0.0.1', user='ssafy', password='ssafy', db='track', charset='utf8')
cursor = conn.cursor()

# DB에서 모든 track id 가져오기
track_ids = []
try:
    query = "SELECT videoUrl FROM track"  # 모든 트랙의 ID를 가져오는 쿼리
    cursor.execute(query)
    results = cursor.fetchall()
    track_ids = [str(result[0]) for result in results]  # 모든 ID를 문자열로 변환하여 리스트에 저장
    if not track_ids:
        print("트랙 ID를 찾을 수 없습니다.")
        exit(1)
except pymysql.MySQLError as e:
    print(f"DB 오류: {e}")
    exit(1)

# 모든 트랙 ID에 대해 반복 작업
for track_id in track_ids:
    # 가져온 track_id 값을 파일명으로 사용 
    desired_title = f"track_{track_id}"  

    # YouTube에서 음악 다운로드
    url = f"https://www.youtube.com/watch?v={track_id}"  # YouTube 링크
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': f'downloaded_music/{desired_title}.%(ext)s',  # 파일 이름에 동영상 제목 포함
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
        'ffmpeg_location': r'D:\ffmpeg\bin'
    }

    # 다운로드
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

    # 오디오 파일 로드
    output_file = f"downloaded_music\{desired_title}.wav"
    if os.path.exists(output_file):
        y, sr = librosa.load(output_file)
        title = video_title
    else:
        print(f"오디오 파일을 찾을 수 없습니다: {output_file}")
        exit(1)

    # 볼륨(Amplitude) 추출
    amplitude = np.abs(librosa.stft(y)).mean(axis=1)

    # 비트 추출
    tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
    beat_times = np.round(librosa.frames_to_time(beat_frames, sr=sr), 2)  # 반올림을 바로 적용

    # 강도 정규화 및 클리핑 (벡터화 연산 사용)
    max_amplitude = max(amplitude)
    intensities = np.clip((amplitude / max_amplitude * 255), 0, 255).astype(int)

    # JSON 데이터 생성
    vibration_data = [{
        "time": beat_times.tolist(),
        "intensity": intensities.tolist()
    }]

    # JSON 파일로 저장
    with open(f"{desired_title}_result.json", "w") as f:
        json.dump(vibration_data, f)
        t2 = time()
        print(f'{t2-t1} 초 소요')

    # 데이터베이스에 time과 intensity 데이터 저장
    try:
        # 배열을 콤마로 구분된 문자열로 변환
        time_string = ','.join(map(str, beat_times))
        intensity_string = ','.join(map(str, intensities))

        # track 테이블에 time과 intensity를 문자열로 저장
        update_query = """
            UPDATE track
            SET time = %s, intensity = %s
            WHERE videoUrl = %s
        """
        cursor.execute(update_query, (time_string, intensity_string, track_id))
        conn.commit()
        t2 = time()
        print(f'{desired_title} 처리 완료. 소요 시간: {t2-t1} 초')
    except pymysql.MySQLError as e:
        print(f"DB 저장 오류: {e}")
        conn.rollback()

conn.close()

[youtube] Extracting URL: https://www.youtube.com/watch?v=jWQx2f-CErU
[youtube] jWQx2f-CErU: Downloading webpage
[youtube] jWQx2f-CErU: Downloading ios player API JSON
[youtube] jWQx2f-CErU: Downloading mweb player API JSON
[youtube] jWQx2f-CErU: Downloading m3u8 information
[info] jWQx2f-CErU: Downloading 1 format(s): 251
[download] Destination: downloaded_music\track_jWQx2f-CErU.webm
[download] 100% of    2.97MiB in 00:00:00 at 6.59MiB/s   
[ExtractAudio] Destination: downloaded_music\track_jWQx2f-CErU.wav
Deleting original file downloaded_music\track_jWQx2f-CErU.webm (pass -k to keep)
track_jWQx2f-CErU 처리 완료. 소요 시간: 5.764209747314453 초
[youtube] Extracting URL: https://www.youtube.com/watch?v=cU0JrSAyy7o
[youtube] cU0JrSAyy7o: Downloading webpage
[youtube] cU0JrSAyy7o: Downloading ios player API JSON
[youtube] cU0JrSAyy7o: Downloading mweb player API JSON
[youtube] cU0JrSAyy7o: Downloading m3u8 information
[info] cU0JrSAyy7o: Downloading 1 format(s): 251
[download] Destination: down

In [2]:
# 유튜브 음원 추출 & 코드 최적화 최종본
import pymysql
import librosa
import yt_dlp
import numpy as np
import requests
import os
from time import time

t1 = time()

# DB 연결
conn = pymysql.connect(host='127.0.0.1', user='ssafy', password='ssafy', db='classik', charset='utf8')
cursor = conn.cursor()

# DB에서 모든 track id 가져오기
track_ids = []
try:
    query = "SELECT video_id FROM track"  # 모든 트랙의 ID를 가져오는 쿼리
    cursor.execute(query)
    results = cursor.fetchall()
    track_ids = [str(result[0]) for result in results]  # 모든 ID를 문자열로 변환하여 리스트에 저장
    if not track_ids:
        print("트랙 ID를 찾을 수 없습니다.")
        exit(1)
except pymysql.MySQLError as e:
    print(f"DB 오류: {e}")
    exit(1)

# 모든 트랙 ID에 대해 반복 작업
for track_id in track_ids:
    # 가져온 track_id 값을 파일명으로 사용 
    desired_title = f"track_{track_id}"  

    # YouTube에서 음악 다운로드
    url = f"https://www.youtube.com/watch?v={track_id}"  # YouTube 링크
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': f'downloaded_music/{desired_title}.%(ext)s',  # 파일 이름에 동영상 제목 포함
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
        'ffmpeg_location': r'C:\Users\SSAFY\Desktop\ffmpeg\bin'
    }

    # 다운로드 예외 처리 추가
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(url, download=True)
            thumbnail_url = info_dict.get("thumbnail")  # 썸네일 URL 가져오기
    except yt_dlp.DownloadError as e:
        print(f"다운로드 오류: {e} - 다음 트랙으로 넘어갑니다.")
        continue  # 오류 발생 시 다음 트랙으로 건너뛰기

    # 썸네일 다운로드
    if thumbnail_url:
        thumbnail_path = f"downloaded_music/{desired_title}_thumbnail.jpg"
        try:
            response = requests.get(thumbnail_url)
            response.raise_for_status()  # 요청이 성공했는지 확인
            with open(thumbnail_path, 'wb') as thumbnail_file:
                thumbnail_file.write(response.content)
            print(f"{desired_title} 썸네일 다운로드 완료.")
        except requests.RequestException as e:
            print(f"썸네일 다운로드 오류: {e}")

    # 오디오 파일 로드
    output_file = f"downloaded_music\{desired_title}.wav"
    if os.path.exists(output_file):
        y, sr = librosa.load(output_file)
    else:
        print(f"오디오 파일을 찾을 수 없습니다: {output_file}")
        exit(1)

    # 볼륨(Amplitude) 추출
    amplitude = np.abs(librosa.stft(y)).mean(axis=1)

    # 비트 추출
    tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
    beat_times = np.round(librosa.frames_to_time(beat_frames, sr=sr), 2)  # 반올림을 바로 적용

    # 강도 정규화 및 클리핑 (벡터화 연산 사용)
    max_amplitude = max(amplitude)
    intensities = np.clip((amplitude / max_amplitude * 255), 0, 255).astype(int)

    # 데이터베이스에 time과 intensity 데이터 저장
    try:
        # 배열을 콤마로 구분된 문자열로 변환
        time_string = ','.join(map(str, beat_times))
        intensity_string = ','.join(map(str, intensities))

        # track 테이블에 time과 intensity를 문자열로 저장
        update_query = """
            UPDATE track
            SET haptic_time = %s, haptic_intensity = %s, thumbnail_url = %s
            WHERE video_id = %s
        """
        cursor.execute(update_query, (time_string, intensity_string, thumbnail_path, track_id))
        conn.commit()
        t2 = time()
        print(f'{desired_title} 처리 완료. 소요 시간: {t2-t1} 초')
    except pymysql.MySQLError as e:
        print(f"DB 저장 오류: {e}")
        conn.rollback()

conn.close()

[youtube] Extracting URL: https://www.youtube.com/watch?v=WShgQkaFlV4
[youtube] WShgQkaFlV4: Downloading webpage


<string>:70: SyntaxWarning: invalid escape sequence '\{'
<>:70: SyntaxWarning: invalid escape sequence '\{'
<string>:70: SyntaxWarning: invalid escape sequence '\{'
<>:70: SyntaxWarning: invalid escape sequence '\{'
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_17912\618157140.py:70: SyntaxWarning: invalid escape sequence '\{'
  output_file = f"downloaded_music\{desired_title}.wav"


[youtube] WShgQkaFlV4: Downloading ios player API JSON
[youtube] WShgQkaFlV4: Downloading mweb player API JSON
[youtube] WShgQkaFlV4: Downloading m3u8 information
[info] WShgQkaFlV4: Downloading 1 format(s): 251
[download] Destination: downloaded_music\track_WShgQkaFlV4.webm
[download] 100% of    2.93MiB in 00:00:00 at 3.98MiB/s   
[ExtractAudio] Destination: downloaded_music\track_WShgQkaFlV4.wav
Deleting original file downloaded_music\track_WShgQkaFlV4.webm (pass -k to keep)
track_WShgQkaFlV4 썸네일 다운로드 완료.
track_WShgQkaFlV4 처리 완료. 소요 시간: 5.427885055541992 초
[youtube] Extracting URL: https://www.youtube.com/watch?v=7hkw4rw3ong
[youtube] 7hkw4rw3ong: Downloading webpage
[youtube] 7hkw4rw3ong: Downloading ios player API JSON
[youtube] 7hkw4rw3ong: Downloading mweb player API JSON
[youtube] 7hkw4rw3ong: Downloading m3u8 information
[info] 7hkw4rw3ong: Downloading 1 format(s): 251
[download] Destination: downloaded_music\track_7hkw4rw3ong.webm
[download] 100% of    5.82MiB in 00:00:01 at 5